# SQLite w/ python

As title, in this I'll go over what I'm learning w/ SQL. It's best to do some notetaking here 

To do list:  
[x]SELECT \
[x]SELECT DISTINCT\
[x]INSERT INTO table\
[x]INSERT INTO table using qmarks\
[x]INSERT INTO table using names \
[x]UPDATING table\
[x]DELETING from table\
[]EXECUTE many commands in one go \
[x]USING WHERE TO SPECIFY CONDITIONS \
[x]AGGREGATE data\
[x]combine WHERE & AGGREGATION\
[]CREATE a inventory project\
[]Use subqueries \
[]Use limit\
[] Use not/<>\
[]use Like\
[]use IN\
[] Between\
[] Order by\ 
[] Alias\
[] Delete a table\
[] use sql functions\
    [] string functions  
        [] lower\
        [] upper\
        [] trim\
        [] substring\
    [] Number functions\
        [] Round\
        [] Floor\
        [] Ceil\
    [] Dates\
        [] Dateadd\
        [] Datediff\
        []DatePart \
        []Format \
        [] GetDate\
        [] Extract\
            [] Year\
            [] Month\
            [] Day\
        [] Is Date\
    [] Logical Functions\
        [] Coalesce\
        [] Nulif\
    [] Conversions\
        [] Cast\
        [] Convert\
[] Add autoIncrement\
[] Edit existing table\
[] Check for null\ 

## Importing sqlite

As a really light and performant application, sqlite is easily added to many things and as a battery included language, python is reliable in that it has sqlite built in. It just needs to be imported

In [2]:
import sqlite3

### Creating a db

All sqlite dbs are files. To create one the syntax is:

In [7]:
import sqlite3

conn = sqlite3.connect('test.db')
cursor = conn.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS people(
  first_name TEXT,
  last_name TEXT,
  age INTEGER
);
""")
conn.commit()
conn.close()

The above statement does multiple things
- creates a connection to the named db `test.db`
- creates a cursor to navigate and control db `cursor = conn.cursor()`
- executes a create table command `cursor.execute("CREATE TABLE IF NOT EXISTS...)
  - IF NOT EXISTS so duplicate tables aren't created
  - columns are named
    - first_name, last_name, age
  - columns are typed
    - TEXT & INTEGER
  - command is ended `;`
- connection is commited to memory `conn.commit()`
- connection is closed `conn.close`

# Accessing table data

The data in tables are accessed with the select command

In [20]:
import sqlite3 
conn =  sqlite3.connect('test.db')
cursor = conn.cursor()
resp = cursor.execute("SELECT * FROM people").fetchall()
conn.commit()
conn.close()

for person in resp: 
    print(person)

('Selena', 'Thomas', 25)
('Johnny', 'Karcol', 34)
('Samantha', 'Grey', 21)


#### Noteworthy things:
- The `execute` command is used to execute sql commands
- `fetchall` is used to tell python to fetch all that matches given conditions
    - otherwise the response will be nothing
    - `fetchone` is an alternative to `fetchall` that fetches the first matching response
    - `fetchall` puts the results in an array
- Data is printed in loop (not very important)
- The connection is **commited** and **closed**

Example of not using fetchall: 

In [23]:
import sqlite3 
conn =  sqlite3.connect('test.db')
cursor = conn.cursor()
resp = cursor.execute("SELECT * FROM people")
conn.commit()
conn.close()
print(resp)

Example of using `fetchone` instead of `fetchall`

In [24]:
import sqlite3 
conn =  sqlite3.connect('test.db')
cursor = conn.cursor()
resp = cursor.execute("SELECT * FROM people").fetchone()
conn.commit()
conn.close()
print(resp)

('Selena', 'Thomas', 25)


- first matching response is printed

## Inserting data into sqlite w/ python

In [28]:
import sqlite3 
conn =  sqlite3.connect('test.db')
cursor = conn.cursor()

cursor.execute("""INSERT INTO people(first_name,last_name,age) --gives structure to insert(optional)
VALUES('Jeremy','Beremy',40)
""") 
response = cursor.execute("SELECT first_name,last_name,age FROM people").fetchall()
print(response)
conn.commit()
conn.close()

[('Selena', 'Thomas', 25), ('Johnny', 'Karcol', 34), ('Samantha', 'Grey', 21), ('Jeremy', 'Beremy', 40)]


#### Noteworthy things: 
- `INSERT INTO` the keyword for inserts
- `people` name of table to insert into
- `people(first_name,last_name,age)` lists the columns to retrieve
    - it's not necessary to list columns
        - instead `INSERT INTO PEOPLE VALUES('Jeremy', 'Beremy',40)` would've been valid
        - Not listing columns runs the risk of having errors
            - the data is given no clear structure so given data in the wrong order could throw an error
            - having the structure makes it so as long as your data follows the given structure, the order isn't restricted
- `fetchall` is used to return an array of matching rows
- connection is commited
- connection is closed

## Alt way of writing connection executions to autoclose connection

In [2]:
import sqlite3 
conn =  sqlite3.connect('test.db')

with conn:
    cursor = conn.cursor()
    print(cursor.execute("SELECT * FROM people").fetchall())
    conn.commit()

[('Selena', 'Thomas', 25), ('Johnny', 'Karcol', 34), ('Samantha', 'Grey', 21), ('Jeremy', 'Beremy', 40)]


## Inserting alts
> This will cover using executemany & an array of values to insert rows
> executemany can prepare and insert multiple values with a given sequence of data

In [8]:
import sqlite3 
conn =  sqlite3.connect('test.db')

with conn: 
    cursor = conn.cursor()
    conn.commit()
     many_inserts = [
        ("Dora", "Explora", 22), 
        ("Cindy", "Dio", 90),
        ("Bob", "Billy", 50)
    ]
    cursor.executemany("""
    INSERT INTO people (first_name,last_name,age)
    VALUES(? ,? ,? )
    """, many_inserts)
    response = cursor.execute("SELECT first_name,last_name,age FROM people").fetchall()
    for person in response:
        print(person)

('Selena', 'Thomas', 25)
('Johnny', 'Karcol', 34)
('Samantha', 'Grey', 21)
('Jeremy', 'Beremy', 40)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)


#### Noteworth things here:
- Inserting many rows at once can be done in python can be done by using the executemany
    - syntax:
     `cursor.executemany(""" INSERT INTO table_name (column1,column2,column3) VALUES(? ,? ,? )""", sequence)`
        - the sequence can be for ex a list of tuples
- `with conn:` is used to open and close the connection
- `executemany` is used to loop through the list to insert all values at once

## Named vs ? 
> The above example used ?s to insert values left to right
> The ex below uses named values instead so the order matters less and the values are pulled from objects rather a tuple or list of data

In [16]:
import sqlite3
conn = sqlite3.connect('test.db')
with conn:
    cursor = conn.cursor()
    many_inserts = [
        {"first_name": "Bruce", "last_name": "Explora","age": 22}, 
        {"first_name": "Glorilla", "last_name": "Dio", "age": 90},
        {"first_name": "Grimes", "last_name": "Billy", "age": 50}
    ]
    cursor.executemany("""INSERT INTO people(first_name,last_name,age) 
        VALUES(:first_name, :last_name, :age) """,many_inserts)
    response = cursor.execute("SELECT first_name, last_name, age FROM people").fetchall()
    for person in response:
        print(person)

('Selena', 'Thomas', 25)
('Johnny', 'Karcol', 34)
('Samantha', 'Grey', 21)
('Jeremy', 'Beremy', 40)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)
('Bruce', 'Explora', 22)
('Glorilla', 'Dio', 90)
('Grimes', 'Billy', 50)


In [17]:
import sqlite3
conn = sqlite3.connect('test.db')
with conn: 
    cursor = conn.cursor()
    response = cursor.execute("SELECT * FROM people").fetchall()
    for person in response: 
        print(person)

('Selena', 'Thomas', 25)
('Johnny', 'Karcol', 34)
('Samantha', 'Grey', 21)
('Jeremy', 'Beremy', 40)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)
('Bruce', 'Explora', 22)
('Glorilla', 'Dio', 90)
('Grimes', 'Billy', 50)


## SELECT DISTINCT(Different) data
> Select distinct is use to only select data that is different from each other

In the ex above, there are dupicates of Dora,Cindy and Bob. Let's try to only print the unique names

In [8]:
import sqlite3 
conn = sqlite3.connect('test.db')
with conn:
    cursor = conn.cursor()
    sel_all = cursor.execute('SELECT first_name,last_name,age FROM people').fetchall()
    sel_dist = cursor.execute('SELECT DISTINCT first_name,last_name,age FROM people').fetchall()
    print("All people:")
    for person in sel_all:
        print(person)
    print("Distinct people")
    for person in sel_dist: 
        print(person)

All people:
('Selena', 'Thomas', 25)
('Johnny', 'Karcol', 34)
('Samantha', 'Grey', 21)
('Jeremy', 'Beremy', 40)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)
('Bruce', 'Explora', 22)
('Glorilla', 'Dio', 90)
('Grimes', 'Billy', 50)
('Samantha', 'Grey', 21)
('Samantha', 'Grey', 21)
('Samantha', 'Grey', 21)
('Samantha', 'Grey', 21)
('Samantha', 'Grey', 21)
('Samantha', 'Grey', 21)
('Samantha', 'Barnes', 25)
('Joe', 'Barnes', 35)
('Sarah', 'Barnes', 25)
Distinct people
('Selena', 'Thomas', 25)
('Johnny', 'Karcol', 34)
('Samantha', 'Grey', 21)
('Jeremy', 'Beremy', 40)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)
('Bruce', 'Explora', 22)
('Glorilla', 'Dio', 90)
('Grimes', 'Billy', 50)
('Samantha', 'Barnes', 25)
('Joe', 'Barnes', 35)
('Sarah', 'Barnes', 25)


### Notes
- Notice how the returned lists are all unique names

In [14]:
import sqlite3 
conn = sqlite3.connect('test.db')
with conn: 
    cursor = conn.cursor()
    last_names = cursor.execute('SELECT DISTINCT last_name FROM people').fetchall()
    first_names = cursor.execute('SELECT DISTINCT first_name FROM people').fetchall()
    ages = cursor.execute('SELECT DISTINCT age FROM people').fetchall()
    print('Unique last names:')
    for person in last_names:
        print(person[0])
    print('\nUnique first names:')
    for person in first_names: 
        print(person[0])

Unique last names:
Thomas
Karcol
Grey
Beremy
Explora
Dio
Billy
Barnes

Unique first names:
Selena
Johnny
Samantha
Jeremy
Dora
Cindy
Bob
Bruce
Glorilla
Grimes
Joe
Sarah


## Updating a table 
> Let's try inserting someone then updating them 

In [2]:
import sqlite3
conn = sqlite3.connect('test.db') 
cursor = conn.cursor()
selena_before = cursor.execute('SELECT * FROM people WHERE first_name="Selena" AND last_name="Thomas"').fetchone()
print(f'{selena_before[0]} {selena_before[1]} is {selena_before[2]} years old')
cursor.execute("""
UPDATE people SET age=69 WHERE first_name="Selena" AND last_name="Thomas";
""")
selena_after = cursor.execute("""
SELECT first_name,last_name,age FROM people WHERE first_name="Selena" AND last_name="Thomas"
""").fetchone()
print(f'{selena_after[0]} {selena_after[1]} is {selena_after[2]} years old')
conn.commit()
conn.close()

Selena Thomas is 40 years old
Selena Thomas is 69 years old


In the above example, Selena is called early to show her age then updated to be 69 rather than 40

Noteworthy things: 
- Selena is targetted specifically using WHERE
    - Selena is specified using her first AND last name using the `AND` keyword
        - the `AND` keyword allows chaining conditions requiring all to be `true`

## SQlite master 
> The sqlite_master is it's own db that contains information about the other table within
> Below we'll create a basic statement to access the tables within  

In [24]:
import sqlite3 
conn = sqlite3.connect('test.db') 
with conn: 
    cursor = conn.cursor()
    response = cursor.execute(""" SELECT type,name,tbl_name,rootpage,sql FROM sqlite_master WHERE type='table';""").fetchall()
    for table in response:
        print(""" 
        Type : {type}
        Table_Name: {tbl_name}
        name : {name}
        rootpage : {rootpage}
        Table Creation statement : {sql}
        """.format(type = table[0], tbl_name = table[2], name = table[1], rootpage = table[3], sql = table[4]) )
    conn.commit()
conn.close()

 
        Type : table
        Table_Name: people
        name : people
        rootpage : 2
        Table Creation statement : CREATE TABLE people(
               first_name TEXT,
               last_name TEXT,
               age INTEGER
)
        
 
        Type : table
        Table_Name: person
        name : person
        rootpage : 3
        Table Creation statement : CREATE TABLE person(
             first_name TEXT,
             last_name TEXT,
             age INTEGER
           )
        


The information above was taken from the sql_master. It provides a good amount of information such as the sql statement used to create the db object, the type of object (in this case all tables), name and table_name of the objects (the same if type of table/view), and the rootpage
> More info on what these all are later

## Deleting from table
> Important to say we know full CRUD

In [52]:
import sqlite3
conn = sqlite3.connect('test.db')
with conn: 
    def deleteFunc():
        deleteCursor = conn.cursor()
        deleteCursor.execute("""DELETE FROM people WHERE first_name='Samantha' AND last_name='Grey' AND age=21 """)
        print('Deleted!')
        conn.commit()
    def getAll():
        getAllCursor = conn.cursor()
        response = getAllCursor.execute('SELECT * FROM people').fetchall()
        for person in response:
            print(person)
            
    cursor = conn.cursor()
    response = cursor.execute('SELECT * FROM people').fetchall()
    print('All data before manipulation')
    for person in response:
        print(person)

     Samanthas = cursor.execute("""
    SELECT * FROM people WHERE first_name='Samantha' AND last_name = 'Grey' AND age=21
    """).fetchall()
    if Samanthas:
        deleteFunc()
    else:
        print('\nFake data needed for deletion. Creating Samantha duplicates...')
        dataToAdd = [
            ('Samantha', 'Grey', 21),
            ('Samantha', 'Grey', 21),
            ('Samantha', 'Grey', 21),
            ('Samantha', 'Grey', 21),
            ('Samantha', 'Grey', 21),
            ('Samantha', 'Grey', 21)
        ]
        print('Fake data: ')
        for samantha in dataToAdd:
            print(samantha)
        print('adding...')
        cursor.executemany("""
        INSERT INTO people(first_name,last_name,age)
        VALUES(?,?,?)
        """,dataToAdd)
        print("\nNew table:")
        getAll()
        print("\nTime to delete!") 
        deleteFunc()
        
    print('\nAfter deletion')
    getAll()

All data before manipulation
('Selena', 'Thomas', 69)
('Johnny', 'Karcol', 34)
('Jeremy', 'Beremy', 40)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)
('Bruce', 'Explora', 22)
('Glorilla', 'Dio', 90)
('Grimes', 'Billy', 50)
('Samantha', 'Barnes', 25)
('Joe', 'Barnes', 35)
('Sarah', 'Barnes', 25)

Fake data needed for deletion. Creating Samantha duplicates...
Fake data: 
('Samantha', 'Grey', 21)
('Samantha', 'Grey', 21)
('Samantha', 'Grey', 21)
('Samantha', 'Grey', 21)
('Samantha', 'Grey', 21)
('Samantha', 'Grey', 21)
adding...

New table:
('Selena', 'Thomas', 69)
('Johnny', 'Karcol', 34)
('Jeremy', 'Beremy', 40)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)
('Bruce', 'Explora', 22)
('Glorilla', 'Dio', 90)
('Grimes', 'Billy', 50)
('Samantha', 'Barnes', 25)
('Joe', 'Barnes', 35)
('Sarah', 'Barnes', 25)
('Samantha', 'Gre

Noteworthy stuff:
- `DELETE FROM table_name` is the general syntax
    - this on it's own would delete all data from the table
    - chaining on `WHERE` w/ conditions specifies data for deletion
- `fetchall()` is always used when fetching more than one cell of data in the table
    - with `fetchone()` used if it's just one

Also I mean this isn't important but I'm feeling pretty comfortable just writing a good amount of python and sql code since I'm still learning both

## Combining AND/OR conditions
> Let's play around a bit with finding data w/ conditions
> I've already done this since it's not very difficult but still

In [78]:
# Print all data first to look at
import sqlite3 
conn = sqlite3.connect('test.db')
with conn: 
    cursor = conn.cursor()
    response = cursor.execute('SELECT * FROM people WHERE last_name ="Barnes" ').fetchall()
    print("All Barnes family members: \n")
    for person in response:
        print(person)
    print("""\n Let's aggregate data by finding the average family member's age \n""")

    print("COUNT, SUM, AVG, MIN, and MAX are some basic aggregation commands available in SQL\n")

    avg_age_result= cursor.execute('SELECT AVG(age) FROM people WHERE last_name="Barnes"').fetchone()
    avg_age = avg_age_result[0]
    print('The average age is {avg_age}\n'.format(avg_age=avg_age))
    print("Let's use this data to find the older and younger siblings\n")
    
    older_siblings = cursor.execute(f'SELECT first_name,age FROM people WHERE age>{avg_age} AND last_name="Barnes"').fetchall()
    younger_siblings = cursor.execute(f'SELECT first_name,age FROM people WHERE age<{avg_age} AND last_name="Barnes"').fetchall()
    if len(older_siblings)> 1:
        print("Older siblings: ")
    else:
        print("Older sibling: ")
    for sibling in older_siblings:
        print(f"{sibling[0]} age {sibling[1]}\n")
    if len(younger_siblings)> 1:
        print("Younger siblings: ")
    else:
        print("Younger sibling: ")
    for sibling in younger_siblings:
        print(f"{sibling[0]} age {sibling[1]}")

All Barnes family members: 

('Samantha', 'Barnes', 25)
('Joe', 'Barnes', 35)
('Sarah', 'Barnes', 25)

 Let's aggregate data by finding the average family member's age 

COUNT, SUM, AVG, MIN, and MAX are some basic aggregation commands available in SQL

The average age is 28.333333333333332

Let's use this data to find the older and younger siblings

Older sibling: 
Joe age 35

Younger siblings: 
Samantha age 25
Sarah age 25


Noteworthy stuff:
- `AVG` combined w/ a `WHERE last_name="Barnes"` is used to specify the average of only the Barnes family

### More aggregations
> Let's do some quick stats

In [102]:
import sqlite3 
conn = sqlite3.connect('test.db')
with conn: 
    cursor = conn.cursor()
    all_people = cursor.execute('SELECT * FROM people').fetchall()
    print('All people:')
    for person in all_people:
        print(person)

    count = cursor.execute('SELECT COUNT(first_name) FROM people').fetchone()
    print('\nAmount of ppl {0}'.format(count[0]))
    avg_age = cursor.execute('SELECT AVG(age) FROM people').fetchone()
    print('\nAverage age of people {}'.format(avg_age[0]))

    youngest = cursor.execute('SELECT * FROM people WHERE age = (SELECT MIN(age) FROM people)').fetchall()
    if len(youngest) < 1:
        print('\nThe youngest person is:')
    else: 
        print('\nThe yougnest peole are: ')
    for youngin in youngest:
        print(youngin)

All people:
('Selena', 'Thomas', 69)
('Johnny', 'Karcol', 34)
('Jeremy', 'Beremy', 40)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)
('Dora', 'Explora', 22)
('Cindy', 'Dio', 90)
('Bob', 'Billy', 50)
('Bruce', 'Explora', 22)
('Glorilla', 'Dio', 90)
('Grimes', 'Billy', 50)
('Samantha', 'Barnes', 25)
('Joe', 'Barnes', 35)
('Sarah', 'Barnes', 25)

Amount of ppl 15

Average age of people 47.6

The yougnest peole are: 
('Dora', 'Explora', 22)
('Dora', 'Explora', 22)
('Bruce', 'Explora', 22)
